### Simple RAG with litesearch
> We will build a simple rag with litesearch. Do not be deceived. We are doing a whole bunch of heavylifting under the hood with very little code.

In [1]:
#| export
from fastcore.all import *
from fastlite import *
import numpy as np
import re
from selectolax.parser import HTMLParser
from litesearch import *

Let's set the db up. This db has usearch loaded. So, you can run cosine distance calculations using simd(means fast, real fast)

In [2]:
db:Database=setup_db('breugel.db')

In [3]:
db.q('select distance_cosine_f16(:vec1,:vec2)', dict(vec1=np.ones(512, np.float16).tobytes(), vec2=np.zeros(512, np.float16).tobytes()))

[{'distance_cosine_f16(:vec1,:vec2)': 1.0}]

There are way more functions you can run now. Checkout: https://unum-cloud.github.io/USearch/sqlite/index.html

#### Ingest PDF documents
> We will ingest a sample PDF documents from Bruegel. We will restrict it to 10 pdfs. But it has about 1800 in total. This isto showcase real world utilities of litesearch.
> We will read the PDF document using `read_pdf` function from litesearch.ingest module.
> We will then scrape the urls from the pdf and then recursively get all pdf's off of those links and ingest them as well.'

In [4]:
#| export
class BruegelDataset:
    ''' Dataset for Bruegel PDF documents.'''
    URL = 'https://www.bruegel.org/system/files/2024-06/Bruegel_factsheet_2024_0.pdf'
    URI_SCHEMA = r'^http://data\.europa\.eu/eli/(?P<typedoc>[^/]+)/(?P<year>\d{4})/(?P<natural_number>\d+)/(?P<date>\d{4}-\d{2}-\d{2})/(?P<lang>[a-z]{2,3})/pdfa2a$'
    def __init__(self, dest:Path=Path('bruegel_dataset')):
        self.dest = dest
        self.pdfs = self()


    @staticmethod
    def _is_pdf_link(l:str): return l.strip() and (l.lower().endswith('.pdf') or 'pdf' in l.lower())
    @staticmethod
    def url2name(url: str) -> str | None:
        if re.match(BruegelDataset.URI_SCHEMA, url):
            m = re.match(BruegelDataset.URI_SCHEMA, url)
            return f"{m['typedoc']}_{m['year']}_{m['natural_number']}_{m['date']}_{m['lang']}.pdf"
        return url.split('/')[-1] if url.split('/')[-1] != '' else url.rstrip('/').split('/')[-1]+'.html'

    def _get_meta(self, r):
        try:
            meta = HTMLParser(r).tags('meta')
            nodes = L([dict2obj(m.attributes) for m in meta]).filter(
                lambda m: 'about' in m
            ).filter(
                lambda m: ('property' in m and m['property'].lower() in ['eli:is_embodied_by','eli:title'])
            ).filter(
                lambda m: ('resource' in m and 'pdfa2a' in m['resource'].lower()) or 'content' in m
            ).groupby('about')
            for k in nodes: nodes[k] = merge(*nodes[k])
            return L([(nodes[n]['resource'], nodes[n]['content']) for n in nodes])
        except: pass


    def read_link(self, l, dest=None):
        try:
	        p = self.save_pdf(l, dest)
	        return p.read_text()
        except: return ''

    def save_pdf(self, l:str, dest:Path=None) -> Path | None:
        try:
            if not l : return None
            if not dest: dest = self.dest
            p = dest / self.url2name(l)
            if not (p.exists() and p.stat().st_size > 1024): p = urlsave(l,p)
            return p
        except Exception as ex: print(ex); return None

    def __call__(self, sample_size=10) -> L:
        '''Make a dataset of documents from a pdf url and all linked pdfs.'''

        def get_linked_pdfs(doc, filter=True):
            links = doc.pages.map(lambda p: p.links.map(lambda l: l.uri)).concat().unique()
            if filter: links = links.filter(self._is_pdf_link)
            return links[:sample_size]

        pth = self.dest / self.url2name(self.URL)
        if not pth.exists(): pth = urlsave(self.URL, self.dest / self.url2name(self.URL))
        main_doc = read_pdf(pth)
        pdf_lp = Path(self.dest / 'pdf_links1')
        if not pdf_lp.exists():
            links = get_linked_pdfs(main_doc, filter=False)
            print(f'Found {len(links)} linked pdfs.')
            all_c = parallel(self.read_link, links, threadpool=True, dest=self.dest/'links')
            pdf_links = parallel(self._get_meta, all_c, threadpool=True).concat().unique()
            print(f'Found {len(pdf_links)} external pdf links.')
            pdf_lp.mk_write('\n'.join([f'{l[0]},{l[1]}' for l in pdf_links]))
        pdf_subset = pdf_lp.readlines()[:sample_size]
        url2tit = {l.split(',')[0]: l.split(',')[1] for l in pdf_subset}
        pdf_list = L([l.split(',')[0] for l in pdf_subset])
        name2url = {self.url2name(l): l for l in pdf_list}
        pdf_set = set(pdf_list.map(self.url2name))
        downloaded_pdfs = globtastic(self.dest / 'pdfs', file_glob='*.pdf', func=Path).filter(lambda m: m.stat().st_size > 1024).map(lambda p: p.name)
        fetch_list = [name2url.get(d) for d in pdf_set.difference(downloaded_pdfs)]
        if fetch_list:
            print(f'Downloading {len(fetch_list)} new pdfs...')
            parallel(self.save_pdf, fetch_list, dest=self.dest/'pdfs')
        return globtastic(self.dest / 'pdfs', file_glob='*.pdf', func=Path).filter(lambda m: m.name in name2url).map(lambda m: AttrDict(path=m, title=url2tit[name2url[m.name]]))

In [5]:
b = BruegelDataset()

KeyboardInterrupt: 

#### Patching is a great way to add functionalities to existing classes. It works on all python classes
Here we are modifying the title in metadata with a better value. We get it as part of the breugel web scraping.

In [8]:
#| export
@patch
def read_pdfs(self:BruegelDataset, fn=noop):
    def update_meta(p):
        if not fn(p): return None
        d = read_pdf(p.path)
        d['name'] = d.metadata.title
        d.metadata.title = p.title
        return d
    return maps(update_meta, self.pdfs)

Let's load the dataset. it takes a bit the first time around as it downloads about 10 pdfs. check your examples folder for `breugel_dataset`

In [9]:
print('no. of pdfs: ', len(b.pdfs))

no. of pdfs:  817


Let's read the first pdf.

In [11]:
doc = first(b.read_pdfs())

Now let's load the pdfs into the db. The db is already patched with a whole bunch of syntactic sugars. Checkout `litesearch.data` for more info

In [12]:
db.pdf_ingest(first(b.read_pdfs()))

/Users/71293/code/thedu/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-09 20:14:50.502 | DEBUG    | chonkie.chunker.base:__init__:32 - Initialized RecursiveChunker
2025-11-09 20:14:50.503 | DEBUG    | chonkie.chunker.recursive:chunk:368 - Starting recursive chunking for text of length 17283
2025-11-09 20:14:50.513 | INFO     | chonkie.chunker.recursive:chunk:370 - Created 10 chunks using recursive chunking
2025-11-09 20:14:52.152 | DEBUG    | chonkie.chunker.base:__init__:32 - Initialized SemanticChunker
2025-11-09 20:14:52.153 | DEBUG    | chonkie.chunker.semantic:chunk:531 - Starting semantic chunking for text of length 2001
2025-11-09 20:14:52.154 | DEBUG    | chonkie.chunker.semantic:chunk:535 - Prepared 40 sentences for semantic analysis
2025-11-09 20:14:52.157 | INFO     | chonkie.c

The above will work, but will load a new chunking pipe everytime. An efficient way is to get the default pipe of create your own. We use chonkie for chunking and embedding. To reduce the number of documents we're also filtering out non english docs. Checkout their docs for more information: https://docs.chonkie.ai/oss

In [13]:
pipe = pdf_pipe()
only_eng = lambda p: p.path.stem.endswith('_eng')
[db.pdf_ingest(p, pipe) for p in b.read_pdfs(fn=only_eng) if p]

2025-11-09 20:16:29.195 | DEBUG    | chonkie.chunker.base:__init__:32 - Initialized RecursiveChunker
2025-11-09 20:16:29.196 | DEBUG    | chonkie.chunker.recursive:chunk:368 - Starting recursive chunking for text of length 16789
2025-11-09 20:16:29.202 | INFO     | chonkie.chunker.recursive:chunk:370 - Created 2 chunks using recursive chunking
2025-11-09 20:16:30.881 | DEBUG    | chonkie.chunker.base:__init__:32 - Initialized SemanticChunker
2025-11-09 20:16:30.881 | DEBUG    | chonkie.chunker.semantic:chunk:531 - Starting semantic chunking for text of length 8252
2025-11-09 20:16:30.882 | DEBUG    | chonkie.chunker.semantic:chunk:535 - Prepared 128 sentences for semantic analysis
2025-11-09 20:16:30.888 | INFO     | chonkie.chunker.semantic:chunk:573 - Created 47 semantic chunks from 128 sentences
2025-11-09 20:16:30.889 | DEBUG    | chonkie.chunker.semantic:chunk:531 - Starting semantic chunking for text of length 8537
2025-11-09 20:16:30.890 | DEBUG    | chonkie.chunker.semantic:chu

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

This can be made even faster with chonie pipelines where you can pass a bunch of texts together and chunk all docs in batches first and push them to db. Checkout https://docs.chonkie.ai/oss/pipelines

Cool, let's search through these contents

In [14]:
content = db.table('content') # can also use db.t.content

In [18]:
fts_search = bind(content.search, order_by='rank', columns=['id','content', 'metadata'], limit=50)

In [19]:
q = 'economic growth in Europe'

In [20]:
fts_r = [r for r in fts_search(q)] # FTS search
print(len(fts_r), 'results got')

4 results got


As you can see, there is no exact term `economic growth in Europe` anywhere in the texts. Let's make the search wide. FTS allows globs like * and use OR. There is so much you can tune with apsw fts. We provide strong defaults (always customisable) with the `pre` function.

In [21]:
fts_r = dict2obj(L(fts_search(pre(q))))
print(len(fts_r), 'results got')

50 results got


This is just scratching the surface, apsw puts fts on steroids. Look forward for more updates to litesearch along these lines. To know more about apsw fts, refer: https://rogerbinns.github.io/apsw/example-fts.html

Okay, We've got some results. Let's get some vec search results.

In [22]:
from chonkie import AutoEmbeddings

Chonkie exposes model2vec embeddings which are static embeddings. Static embeddings are 500 times smaller and 50 times faster with a very small performance loss. Potion-retrieval-32M is a good model imo. Learn more at: https://github.com/MinishLab/model2vec

In [23]:
embedding_fn = AutoEmbeddings().get_embeddings('minishlab/potion-retrieval-32M').embed

In [25]:
vec_r = L(dict2obj(content(select='id, content, metadata', where='embedding is not null', where_args=dict(qvec=embedding_fn(q).tobytes()), order_by='distance_cosine_f32(embedding, :qvec)',limit=50)))
print(len(fts_r), 'results got')

50 results got


Cool, we see results. Now, these are ways to get these results separately. The underlying fastlite and apswutils wrappers over sqlite are powerful and allow you to manipulate the db in efficient ways. Checkout: https://github.com/AnswerDotAI/fastlite and https://github.com/AnswerDotAI/apswutils/tree/main

#### litesearch provides a search method which reranks the results from both FTS and vector search using Reciprocal Rank Fusion (RRF)
> You can always turn it off.

In [26]:
res=db.search(pre(q), embedding_fn(q).tobytes(), columns=['id', 'content', 'metadata'], rrf=True)

In [36]:
print(first(res))

{'id': 28599, 'content': 'Property Rights (IPR) appli\xad\ncations \nInnovations - \nNumber of innovations \nresulting from the projects \nfunded by the Programme \n(by type of innovation) \nincluding from awarded IPRs \nEconomic growth - \nCreation, growth & market \n', 'metadata': '{"tokens": 79, "start_index": 189182, "end_index": 189366, "context": "Economic growth - \\nCreation, growth & market \\n", "name": "CL2021R0695EN0010010.0001.3bi_cp 1..1", "path": null, "format": "PDF 1.7", "title": " Regulation (EU) 2021/695 of the European Parliament and of the Council of 28\xa0April 2021 establishing Horizon Europe – the Framework Programme for Research and Innovation", "author": "Publications Office", "subject": " ", "keywords": "", "creator": "Arbortext Advanced Print Publisher 10.0.1465/W Unicode", "producer": "3-Heights(TM) PDF to PDF-A Converter Shell 4.7.24.2 (http://www.pdf-tools.com)", "creationDate": "D:20240320171849+05\'00\'", "modDate": "D:20240326180552+01\'00\'", "trapped

You do not need to, but you can now flashrank these results if needed. The Potion-retrieval model is a bge distilled model which is a fusion cross encoder with strong retrieval capabilities.

In [28]:
from flashrank import Ranker, RerankRequest

It's a good idea to understand what the mak token limit is for the documents in the db so that we can set the max length of the ranker accordingly. If you set a high amount, it does affect performance.So, the ideal is to set it to max tokens + 100-200

In [30]:
max_token = int(db.q("select max(json_extract(metadata, '$.tokens')) as m from content")[0]['m'])

In [31]:
ranker = Ranker(max_length=max_token+150)

In [33]:
res1=ranker.rerank(RerankRequest(q, res.map(lambda r: dict(text=r.content, id=r.id))))

In [37]:
print(first(res1))

{'text': "\nTo address future societal challenges, embrace the opportunities of new tech\xad\nnologies and contribute to environmentally friendly and sustainable economic \ngrowth, jobs, competitiveness and the well-being of Europe's citizens, there is \nthe need to further strengthen Europe's capacity to innovate by: strengthening \n", 'id': 28340, 'score': np.float32(0.99480695)}


Now, let's compare flashrank results and litesearch rerank results.

In [35]:
[print(r1['id'], r2['id']) for r1,r2 in zip(res,res1)]

28599 28340
25833 25394
25834 25163
25335 26874
25163 25834
28600 25829
25393 28263
28282 28600
25394 26895
25173 28262
28595 25170
25164 24751
25829 25173
26175 26718
25336 23607
28283 27144
25821 23218
28340 25336
27813 25164
25169 21238
23607 25315
26874 23605
25170 28599
24751 24528
28477 23613
26905 28282
28262 25821
26873 27813
25737 22904
28376 25833
5785 25738
25738 28595
9283 26189
28597 13644
26853 29289
5971 28215
28263 10742
21165 1302
5972 25335
21164 28214
9435 25169
28478 28283
26906 10741
13906 1939
28598 25393
32650 24473
26884 24276
1538 26894
26854 26906
25315 26151
27144 26175
26895 28478
10742 25737
13644 26853
10741 27737
21238 28477
29289 23287
24473 25775
26151 23608
27737 23311
1939 22965
24276 26854
22904 28598
23613 30101
26894 1538
23311 26905
23608 26884
23605 15307
25775 31871
1302 26873
30101 5785
23287 27193
27193 28597
28215 21165
31871 9435
10797 10208
26718 5972
15307 9283
23218 10797
10208 32650
24528 31870
31870 5971
28214 28376
22965 13906
26189 21

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

So you have it. a simple rag pipeline.